In [16]:
import psycopg2 as ps
import pandas as pd
import Preprocessing
import time
import numpy as np
from collections import Counter
import fuzzywuzzy as fuz

## Getting first time data

In [ ]:
## Connect to database

conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
cursor = conn.cursor()

q = '''SELECT * FROM TWEETS
        WHERE TWEET_CREATED >= '2018-01-01' 
        AND TWEET_CREATED < '2021-01-01'; '''
cursor.execute(q)
results = cursor.fetchall()
colnames = [desc[0] 
        for desc in cursor.description]
cursor.close()
conn.close()

df = pd.DataFrame(results, columns=colnames)

In [4]:
start = time.time()
df_clean = Preprocessing.clean_text(df, 'tweet_text')
end = time.time()
print(str(end-start), 'ms')

11.453316688537598 ms


In [5]:
df_clean.clean_text = df_clean.clean.copy()
df_clean = df_clean.drop('clean', axis=1)

In [6]:
# shift column 'Name' to first position
clean_text = df_clean.pop('clean_text') 
# insert column using insert(position,column_name,
# first_column) function
df_clean.insert(6, 'clean_text', clean_text)

## Using data csv

In [99]:
short_df = df_clean[['tweet_id', 'clean_text', 
    'isincident', 'isaccident', 'isobstacle', 'isdanger', 'incident_desc']].copy()

In [17]:
short_df = pd.read_csv('tweets-2018-2020-clean.csv')
short_df.drop('Unnamed: 0', axis=1, inplace=True)
## Fill NaN with None (if condition is false, then None)
short_df = short_df.where(pd.notnull(short_df), None)

In [4]:
short_df

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
0,1013816364220518402,precaucion calle 50 anegada via,None,None,None,None,None,None
1,951132392906870784,recorrido mañanitas,None,None,None,None,None,None
2,1010612149826342914,inundada calle 70 san fransisco,None,None,None,None,None,None
3,1013816934142562304,inundada calle 74 san francisco precaucion ruta,None,None,None,None,None,None
4,1022561878969671680,inundada avenida frente mercado marisco,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
87101,1344759940305612820,5km h desplazan autos via transistmica altura ...,None,None,None,None,None,None
87102,1344773829290287105,recibe resumen noticias capsulas video canal y...,None,None,None,None,None,None
87103,1344780610594856967,celebremos viven casa forma virtual expresemos...,None,None,None,None,None,None
87104,1344810167276363781,registra accidente transito via centenario alt...,True,True,False,False,accidente,None


In [5]:
len(short_df)

87106

In [286]:
## Count values still not analyzed
len(short_df[short_df.isincident.isnull()])

24412

In [10]:
pd.set_option('display.max_columns', 500)
pd.set_option("max_colwidth", 200)
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 100)

In [284]:
## Check the true with the keyword
extract = short_df.clean_text.str.contains('altura') & short_df.isincident.isnull()
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
114,948308001710100482,auto avenida martires altura asamblea nacional direccion panama oeste precaucion,None,None,None,None,None,None
346,949709628547444743,camion derramo piedra tramo via centenario altura puente direccion panama centro via noticias,None,None,None,None,None,None
429,950182593940598784,balacera via españa altura balboa policia nacional realiza persecucion via,None,None,None,None,None,None
433,950190920942673920,balacera via españa altura balboa video,None,None,None,None,None,None
434,950194454073405441,mantiene accion policial via españa altura puente balboa persecucion balacera reporto evitar area,None,None,None,None,None,None
435,950197694135402498,balacera via españa altura balboa policia nacional realiza persecucion evitar area via,None,None,None,None,None,None
536,950812241762320384,desprende contenedor mula via transistmica altura san miguelito direccion panama norte,None,None,None,None,None,None
666,951575257231618050,camion derriba tendido electrico afecta inversion carriles realizan ingresar arraijan cabecera altura policlinica arraijan,None,None,None,None,None,None
671,951583504332214272,camion derriba tendido electrico afecta inversion carriles realizan ingresar arraijan cabecera altura policlinica arraijan,None,None,None,None,None,None
674,951589114045968389,camion derriba tendido electrico afecta inversion carriles realiza ingresar arraijan cabecera altura policlinica arraijan via,None,None,None,None,None,None


In [283]:
counts = Counter(' '.join(short_df[short_df.clean_text.notnull() & short_df.isincident.isnull()].clean_text).split())
# short_df.isaccident.isnull() & short_df.clean_text.notnull()
freq = pd.DataFrame(counts.most_common(20), columns=['word', 'count'])
freq

,word,count
0,via,5012
1,panama,4890
2,altura,3021
3,precaucion,2280
4,nacional,1918
5,direccion,1659
6,policia,1542
7,san,1489
8,tu,1414
9,dia,1400


# Accidentes

In [276]:
len(short_df[extract])

230

In [38]:
counts = Counter(' '.join(short_df[extract].clean_text).split())
# short_df.isaccident.isnull() & short_df.clean_text.notnull()
freq = pd.DataFrame(counts.most_common(20), columns=['word', 'count'])
freq.head(5)

,word,count


In [277]:
short_df.loc[extract, 'isincident'] = True
short_df.loc[extract, 'isaccident'] = True
# short_df.loc[extract, 'isobstacle'] = False
# short_df.loc[extract, 'isdanger'] = False
short_df.loc[extract, 'incident_desc'] = 'accidente'

In [275]:
extract = short_df.clean_text.str.contains(' incidente ') & short_df.isincident.isnull()
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
361,949745299286843398,mula sufre incidente avenida jose agustin arango altura cruce mañanitas precaucion,None,None,None,None,None,None
376,949768773426434051,mantiene mula sufrio incidente avenida jose agustin arango altura cruce mañanitas precaucion,None,None,None,None,None,None
377,949775908017238016,mantiene mula sufrio incidente avenida jose agustin arango altura cruce mañanitas via,None,None,None,None,None,None
657,951549924814082049,camion deja caer piedras subida banco general 12 octubre manejar precaucion ruta presento incidente,None,None,None,None,None,None
706,951815826734288896,reportan incidente policial calle 2 direccion investigacion judicial dij provincia colon persona estaria herida producto intercambio disparos area video,None,None,None,None,None,None
1166,954126453116940288,registra incidente avenida 12 octubre altura banco nacional precaucion ruta,None,None,None,None,None,None
1310,954882384071184384,pronuncia incidente taxista usuaria reportado redes sociales,None,None,None,None,None,None
1317,955090487060389888,familiares joven perdio vida disparo confuso incidente involucro unidad caimitillo cierran via pedir justicia via,None,None,None,None,None,None
1322,955139875476590592,unidades policia nacional involucradas incidente murio joven 19 años chilibre,None,None,None,None,None,None
1324,955143893749821440,institucion lamentamos ocurrio declara comisionado vega pino policia nacional incidente ocurrido chilibre fallecio joven 19 años,None,None,None,None,None,None


In [336]:
## Specific rows
short_df.loc[753, 'isdanger'] = False

In [ ]:
## Save data into the same CSV (override)
short_df.to_csv('tweets-2018-2020-clean.csv', encoding='utf-8')

In [208]:
## Check the true with the keyword
extract = short_df.clean_text.str.contains('accidente[s]?') & short_df.isaccident==True
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
14,947883288630300673,registra accidente transito autopista arraijan chorrera altura bique direccion interior pais,True,True,False,False,accidente,None
15,947885261849071616,imagenes accidente transito autopista arraijan chorrera altura bique direccion interior pais,True,True,False,False,accidente,None
17,947891590172901383,trafico afectado accidente transito autopista arraijan chorrera altura bique direccion interior pais,True,True,False,False,"tranque, accidente",None
18,947896257024004097,registra accidente transito autopista arraijan chorrera altura bique direccion interior via,True,True,False,False,accidente,None
19,947897042386505728,accidente transito autopista arraijan chorrera altura bique direccion interior pais via,True,True,False,False,accidente,None
20,947903020901257216,mantiene trafico afectado accidente transito autopista arraijan chorrera altura bique direccion interior pais,True,True,False,False,"tranque, accidente",None
21,947903052320858112,aguaceros city panama cuidado calles suficientes accidentes muertos cuidado 1 50pm,True,True,None,None,accidente,None
22,947904579659816960,heridos deja accidente transito carretera costa altura villa luisa provincia colon,True,True,False,False,accidente,None
24,947912385658064898,accidente transito autopista arraijan chorrera altura bique direccion interior pais via noticias,True,True,False,False,accidente,None
25,947912884964687872,accidente transito carretera costa altura villa luisa provincia colon deja 5 heridos,True,True,False,False,accidente,None


In [15]:
short_df.loc[extract, 'isincident'] = False
short_df.loc[extract, 'isaccident'] = False
# short_df.loc[extract, 'isobstacle'] = False
# short_df.loc[extract, 'isdanger'] = False
# short_df.loc[extract, 'incident_desc'] = None

# Inspect

In [372]:
## Inspect a certain keyword in CSV
short_df[extract].to_csv('colision-transito.csv', encoding='utf-8')

In [381]:
revision = pd.read_csv('colision-transito.csv')
revision = revision.where(pd.notnull(revision), None)

In [399]:
len(revision)

394

In [400]:
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==1].tweet_id), 'isaccident'] = True
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==0].tweet_id), 'isaccident'] = False
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==1].tweet_id), 'isincident'] = True
# short_df.loc[short_df.tweet_id.isin(revision[revision.isaccident==1].tweet_id), 'incident_desc'] = 'colision'
short_df.loc[short_df.tweet_id.isin(revision.tweet_id), :]

# Peligros

In [264]:
short_df.loc[extract, 'isincident'] = True
# short_df.loc[extract, 'isaccident'] = True
short_df.loc[extract, 'isdanger'] = True
# short_df.loc[extract, 'isobstacle'] = True
short_df.loc[extract, 'incident_desc'] = 'incendio'

In [263]:
extract = short_df.clean_text.str.contains('^incendi.*.*') & short_df.isincident.isnull()
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
3237,965348021167894528,incendio masa vegetal area crisol residencias afectadas humo bomberos,None,None,None,None,None,None
3240,965349672834555905,incendio masa vegetal area crisol residencias afectadas humo bomberos via,None,None,None,None,None,None
3408,966358451956928512,incendio casco antiguo,None,None,None,None,None,None
3623,967382174822490112,incendio acceder bomberos ph buona vista requieren ancho minimo 6 1 mts envien ingenieros midan certifiquen obra cumple,None,None,None,None,None,None
3946,968949786954039296,incendio masa vegetal mallorca park ojo agua unidades cuerpo bomberos realizan labores extincion,None,None,None,None,None,None
4469,971074187803725824,incendio local estacion metro pueblo via,None,None,None,None,None,None
4566,971461614049382400,incendio masa vegetal avenida domingo diaz altura villa lucre bomberos,None,None,None,None,None,None
5322,974757759571910656,incendio masa vegetal altura burunga via centenario direccion panama oeste,None,None,None,None,None,None
5326,974766620534104065,incendio masa vegetal via centenario altura burunga direccion panama oeste via,None,None,None,None,None,None
5496,975730049692336128,incendio grua dorado,None,None,None,None,None,None


# Obstáculos

In [270]:
# len(short_df[short_df.isincident.isnull()])
len(short_df[extract])

67

In [285]:
counts = Counter(' '.join(short_df[extract].clean_text).split())
# short_df.isaccident.isnull() & short_df.clean_text.notnull()
freq = pd.DataFrame(counts.most_common(20), columns=['word', 'count'])
freq

,word,count
0,altura,3021
1,via,1960
2,direccion,1283
3,panama,1203
4,precaucion,792
5,centro,584
6,avenida,578
7,ruta,474
8,interamericana,435
9,mantiene,374


In [281]:
short_df.loc[extract, 'isincident'] = True
# short_df.loc[extract, 'isaccident'] = False
short_df.loc[extract, 'isobstacle'] = True
short_df.loc[extract, 'isdanger'] = False
short_df.loc[extract, 'incident_desc1'] = 'cierre'

In [280]:
extract = short_df.clean_text.str.contains('^cierr.*') & short_df.isobstacle.isnull()
short_df[extract]

,tweet_id,clean_text,isincident,isaccident,isobstacle,isdanger,incident_desc,incident_desc1
709,951860680663228418,cierre calle ambas direcciones via españa altura iglesia piedra rio abajo evitar ruta,None,None,None,None,None,None
710,951862663616913408,cierre calle ambas direcciones via españa altura iglesia piedra rio abajo evitar ruta via,None,None,None,None,None,None
1315,955089092181680128,cierre calle amiliares allegados michelle batista 20 años pidiendo justicia murio disparo persecucion policial portal casa caimitillo via,None,None,None,None,None,None
2325,960653037831442434,cierre via interamericana altura entrada lomas mastranto chorrera via,None,None,None,None,None,None
3016,964136046962335744,cierre calle altura super 99 rio abajo utilizar vias alternas,None,None,None,None,None,None
3017,964137884445011968,cierre calle grupo personas via españa supermercado 99 rio abajo,None,None,None,None,None,None
3020,964160669800194048,cierre calle grupo personas via españa super 99 rio abajo via,None,None,None,None,None,None
3024,964173627590692864,cierre corredor norte panama colon rumbo panama altura kuna nega,None,None,None,None,None,None
4107,969643346393292800,cierran cellphone parking abren dejan estacionar buses turismo carajo casco viejo,None,None,None,None,None,None
4361,970668514024534018,cierre calle grupo padres familias via principal veracruz,None,None,None,None,None,None


# Save data

In [287]:
## Save data into the same CSV (override)
short_df.to_csv('tweets-2018-2020-clean.csv', encoding='utf-8')

In [9]:
# for i in range(2014, 2021):
#     df_clean[(df_clean.tweet_created>='%i-01-01'%i) & (df_clean.tweet_created<'%i-01-01'%(i+1))].to_csv('tweets_%i.csv'%(i), encoding='utf-8')
df_clean.to_csv('tweets-2018-2020.csv', encoding='utf-8')